# NMF matching to compare profiles
The steps of this algorithm
## Figuring out the representation of each profile
- Raw components are extracted using NMF on the days of each profile separately  
- Problem: the components don't have a scale
    - look into the coefficient matrix at all the non-zero coefficients that are used with this component 
    - make a kernel density estimation of these coefficients (just fixed bandwidth) and look for the local maxima in this density estimation 
    - these local maxima will be the scales that we use (so here we introduce an error) 
    - now each component that has multiple local maxima will be split up in different components each with one local maxima 
    - in this way we can solve the scale issue 
- So now we have the representation a set of (scaled) components, each component also keeps track of how much it is used in the profile 

## Comparing different profiles 
To compare different profiles, we are going to try to match the consumption with each other.  
The distance between two profiles is then the total consumption that couldn't be matched under time warping.  
This is implemented by greedily matching consumption untill no matches can be made anymore.  


_note: the implementation is pretty expensive, it is iterative and lots of DTW distances need to be calculated_  
But these are worries for later

# Imports

In [ ]:
# this reloads code from external modules automatically 
%load_ext autoreload
%autoreload 2

from KDEpy import TreeKDE
import altair as alt
import numpy as np
import pandas as pd
from pathlib import Path
import itertools
import datetime
from sklearn_extra.cluster import KMedoids
from sklearn.decomposition import NMF
idx = pd.IndexSlice
alt.data_transformers.disable_max_rows()
import warnings
from component_matching import (
    get_component_similarity,
    get_day_df,
    get_NMF,
    scale_components_simple,
    scale_components_discrete, 
    add_date
)
from tqdm import tqdm

# Read (subset of) the data

In [ ]:
PRE_PATH = Path('/cw/dtaiproj/ml/2020-FLAIR-VITO/profile-clustering/preprocessed/combined')
info_path = PRE_PATH/'reindexed_info.csv'
data_path = PRE_PATH/'reindexed_DST_data.csv'
info_df = pd.read_csv(info_path, index_col = [0,1])
data_df = pd.read_csv(data_path, index_col = [0,1], nrows = 100)
data_df.columns = pd.to_datetime(data_df.columns)
data_df.columns.name = 'timestamp'


## Data subset to cluster

In [ ]:
info_df.shape[0]

In [ ]:
data_df = data_df.sample(20, random_state = 123)

## Calculate the distance matrix

In [ ]:
nb_meters = data_df.shape[0]
distance_matrix = np.zeros((nb_meters, nb_meters))
for i,j in tqdm(itertools.product(range(0,nb_meters), range(0, nb_meters)), total = nb_meters*nb_meters): 
    distance_matrix[i,j]= get_component_similarity(data_df.iloc[i], data_df.iloc[j])
distance_matrix

In [ ]:
pd.DataFrame(distance_matrix)

## Cluster the profiles using this metric

In [ ]:
clusterer = KMedoids(n_clusters = 2, metric = 'precomputed').fit(distance_matrix)
labels = pd.Series(clusterer.labels_, index = data_df.index)
labels

## Cluster sizes

In [ ]:
(
    labels
    .value_counts()
    .to_frame('count')
    .rename_axis(index = 'cluster_nb')
    .sort_index()
)

## Show the clusters

In [ ]:
profiles_to_show = labels.index[labels == 0]



In [ ]:
def show_profiles_of_cluster(cluster_idx): 
    profiles_to_show = labels.index[labels == cluster_idx]
    plot_df = (
        data_df.loc[profiles_to_show]
        # drop the year level
        .droplevel(1)
        .stack()
        .to_frame('value')
        .reset_index()
        .assign(
            time = lambda x: add_date(x.timestamp.dt.time),
            date = lambda x: x.timestamp.dt.date.astype('str')
        )
    )
    return alt.Chart(plot_df, width = 600, height = 300).mark_line(size = 0.3).encode(
        x = 'time:T', 
        y= 'value', 
        color = alt.Color('date', scale = alt.Scale(scheme = 'rainbow'), legend = None),
    ).facet(row = 'meterID')

In [ ]:
show_profiles_of_cluster(0)

In [ ]:
show_profiles_of_cluster(1)

# Choose two example profiles to work with

In [ ]:
# IDX1, IDX2 = 0,1 #pretty similar
# IDX1, IDX2 = 1,2 # similar
IDX1, IDX2 = 1,4 # similar

profile1 = data_df.iloc[IDX1]
profile2 = data_df.iloc[IDX2]
day_df1 = get_day_df(profile1)
day_df2 = get_day_df(profile2)

In [ ]:
def all_day_chart(day_df): 
    return 

In [ ]:
all_day_chart(day_df1).properties(title = 'Profile 1') & all_day_chart(day_df2).properties(title = 'Profile 2')

# Calculate the NMF of each profile

In [ ]:
get_component_similarity(profile1, profile2)

In [ ]:
get_component_similarity(profile2, profile1)

# Handle the scale 

### Distribution plot of the coefficients of each component

In [ ]:
# each component is used a couple of times with a large coefficient
dfs = []
for component_nb in components_df1.index:
    component_values = representation_df1[component_nb].pipe(lambda x: x[x>0.03])
    x, y = TreeKDE(bw = 0.01).fit(component_values.values).evaluate()
    kde_df = (
        pd.DataFrame()
        .assign(
            x = x, 
            y = y, 
            component_nb = component_nb
        )
    )
    dfs.append(kde_df)
    
all_kde_dfs = pd.concat(dfs, axis = 0)
alt.Chart(all_kde_dfs, title = 'distribution of the coefficients of each component').mark_line().encode(
    x = 'x', 
    y = 'y',
    color = 'component_nb:N'
)

### For now just use the maximum of the KDE (in some way the most common coefficient)


In [ ]:
def scale_components(representation_df, components_df): 
    dfs = []
    for component_nb in components_df.index:
        component_values = representation_df[component_nb].pipe(lambda x: x[x>0.03])
        x, y = TreeKDE(bw = 0.01).fit(component_values.values).evaluate()
        kde_df = (
            pd.DataFrame()
            .assign(
                x = x, 
                y = y, 
                component_nb = component_nb
            )
        )
        dfs.append(kde_df)

    all_kde_dfs = pd.concat(dfs, axis = 0)
    most_common_coefficients = all_kde_dfs.groupby('component_nb')[['y', 'x']].max()['x']
    scaled_components = components_df.multiply(most_common_coefficients, axis = 0)
    times_used = (representation_df > 0.03).sum(axis = 0)
    return scaled_components, times_used

In [ ]:
scaled_components_df1, times_used1 = scale_components(representation_df1, components_df1)
scaled_components_df2, times_used2 = scale_components(representation_df2, components_df2)

In [ ]:
alt.hconcat(all_day_chart(day_df1).properties(title = 'all days'), NMF_component_chart(scaled_components_df1).properties(title = 'scaled components'), NMF_component_chart(components_df1).properties(title = 'non scaled components')).resolve_scale(color = 'independent', y = 'shared').display()

In [ ]:
alt.hconcat(all_day_chart(day_df2).properties(title = 'all days'), NMF_component_chart(scaled_components_df2).properties(title = 'scaled components'), NMF_component_chart(components_df2).properties(title = 'non scaled components')).resolve_scale(color = 'independent', y = 'shared').display()

# Matching algorithm
The goal here is to compare two sets of scaled components which each have a time used

In [ ]:
(NMF_component_chart(scaled_components_df1) | NMF_component_chart(scaled_components_df2)).resolve_scale(color = 'independent', y = 'shared')

## Calculate all the aligned sequences and DTW distances

In [ ]:

from dtaidistance import dtw
import itertools

In [ ]:
class ComponentMatcher:
    """
        A class to help keep track of all the necessary information to make the matching algorithm pretty simple
    """
    
    def __init__(self,scaled_components_df1, times_used1, scaled_components_df2, times_used2): 
        # keeps track of the times used attribute of each component
        self.times_used_dict = dict()
        # keeps track of all the aligned sequences
        self.aligned_sequence_dict = dict()
        # keeps track of all the dtw distances
        self.dtw_distances_dict = dict()
        
        # keeps track of the original components 
        self.original_components1 = {key: value.to_numpy() for key, value in scaled_components_df1.iterrows()}
        self.original_components2 = {key: value.to_numpy() for key, value in scaled_components_df2.iterrows()}
        
        
        
        
        # integers to use for the next components 
        self.current_component_nb1 = scaled_components_df1.index.max()
        self.current_component_nb2 = scaled_components_df2.index.max()
        
        # all components in set1
        self.current_components1 = set(scaled_components_df1.index)
        self.current_components2 = set(scaled_components_df2.index)
        
        # initialise the aligned sequence dict and dtw distances dict
        for comp_nb1, comp_nb2 in itertools.product(scaled_components_df1.index, scaled_components_df2.index):
            component1 = scaled_components_df1.loc[comp_nb1].to_numpy()
            component2 = scaled_components_df2.loc[comp_nb2].to_numpy()
            # added penalty to ensure no warping is preffered over warping 
            aligned_component1, best_path = dtw.warp(component1, component2, window = 4, penalty = 0.1)
            dist = dtw.distance(component1, component2, window = 4, penalty = 0.1)
            self.aligned_sequence_dict[(comp_nb1, comp_nb2)] = (aligned_component1, component2, best_path)
            self.dtw_distances_dict[(comp_nb1, comp_nb2)] = dist
        
        # initialise the times_used_dict 
        for index, value in times_used1.iteritems(): 
            self.times_used_dict[(1,index)] = value 
        for index, value in times_used2.iteritems(): 
            self.times_used_dict[(2,index)] = value 
    
    def one_set_empty(self): 
        return len(self.current_components1)==0 or len(self.current_components2)==0
    
    def get_non_empty_component_set(self): 
        assert self.one_set_empty()
        if len(self.current_components1) == 0: 
            return [self.original_components2[idx] for idx in self.current_components2]
        return [self.original_components1[idx] for idx in self.current_components1]
        
    def get_best_aligned_pair(self): 
        best_pair = min(self.dtw_distances_dict, key = self.dtw_distances_dict.get)
        component1, component2, best_path = self.aligned_sequence_dict[best_pair]
        times_used1 = self.times_used_dict[(1, best_pair[0])]
        times_used2 = self.times_used_dict[(2, best_pair[1])]
        return (best_pair[0], component1, times_used1), (best_pair[1], component2, times_used2), best_path
    
    def remove_component_from_set1(self,comp1): 
        component1_pairs = [(comp1, other_comp2) for other_comp2 in self.current_components2]
        for pair_to_remove in component1_pairs: 
            self.aligned_sequence_dict.pop(pair_to_remove)
            self.dtw_distances_dict.pop(pair_to_remove)
        
        self.times_used_dict.pop((1, comp1))
        
        self.current_components1.remove(comp1)
        
    def remove_component_from_set2(self,comp2): 
        component2_pairs = [(other_comp1, comp2) for other_comp1 in self.current_components1]
        for pair_to_remove in component2_pairs: 
            self.aligned_sequence_dict.pop(pair_to_remove)
            self.dtw_distances_dict.pop(pair_to_remove)
            
        self.times_used_dict.pop((2, comp2))
        
        self.current_components2.remove(comp2)
        
    def add_component_to_set1(self,component1, times_used): 
        comp1_idx = self.current_component_nb1 + 1
        self.current_component_nb1 += 1
        self.current_components1.add(comp1_idx)
        
        self.original_components1[comp1_idx] = component1
        
        self.times_used_dict[(1,comp1_idx)] = times_used
        
        for comp2_idx in self.current_components2: 
            component2 = self.original_components2[comp2_idx] 
            aligned_component1, _ = dtw.warp(component1, component2, window = 4, penalty = 0.1)
            dist = dtw.distance(component1, component2, window = 4, penalty = 0.1)
            self.aligned_sequence_dict[(comp1_idx, comp2_idx)] = (aligned_component1, component2)
            self.dtw_distances_dict[(comp1_idx, comp2_idx)] = dist
            
   
        
    def add_component_to_set2(self,component2, times_used): 
        comp2_idx = self.current_component_nb2 + 1
        self.current_component_nb2 += 1
        self.current_components2.add(comp2_idx)
        
        self.original_components2[comp2_idx] = component2
        
        self.times_used_dict[(2,comp2_idx)] = times_used
        
        for comp1_idx in self.current_components1: 
            component1 = self.original_components1[comp1_idx] 
            aligned_component1, _ = dtw.warp(component1, component2, window = 4, penalty = 0.1)
            dist = dtw.distance(component1, component2, window = 4, penalty = 0.1)
            self.aligned_sequence_dict[(comp1_idx, comp2_idx)] = (aligned_component1, component2)
            self.dtw_distances_dict[(comp1_idx, comp2_idx)] = dist
    
    def change_times_used_set1(self,comp1_idx, times_used): 
        self.times_used_dict[(1, comp1_idx)] = times_used
        
    def change_times_used_set2(self,comp2_idx, times_used): 
        self.times_used_dict[(2, comp2_idx)] = times_used 
        

In [ ]:
component_matcher = ComponentMatcher(scaled_components_df1, times_used1, scaled_components_df2, times_used2)
while not component_matcher.one_set_empty(): 
    
    (comp1_idx, component1, used1), (comp2_idx, component2, used2), warping_path = component_matcher.get_best_aligned_pair()

    print(f"Pair {comp1_idx}, {comp2_idx} used {used1}, {used2}")
    if used1 > used2: 
        diff = used1-used2
        # add component 1 with times_used diff 
        component_matcher.change_times_used_set1(comp1_idx, diff)
        component_matcher.remove_component_from_set2(comp2_idx)
    elif used2 < used1: 
        diff = used2-used1
        # add component 2 with times_used diff 
        component_matcher.change_time_used_set2(comp2_idx, diff)
        component_matcher.remove_component_from_set1(comp1_idx)
    else: 
        component_matcher.remove_component_from_set1(comp1_idx)
        component_matcher.remove_component_from_set2(comp2_idx)
        
    used = min(used1, used2)

    difference = component1 - component2
    positive_sum = np.sum(difference[difference>0])
    negative_sum = -np.sum(difference[difference<0])
    if positive_sum > negative_sum: 
        # comp1 is the one to use 
        new_comp1 = difference
        new_comp1[new_comp1 < 0] = 0
        # add new comp1 with times_used used 
        component_matcher.add_component_to_set1(new_comp1, used)
        distance += negative_sum
    else: 
        new_comp2 = -difference
        new_comp2[new_comp2 < 0] = 0
        # add new comp2 with times_used used 
        component_matcher.add_component_to_set2(new_comp2, used)
        distance += positive_sum 
left_over = component_matcher.get_non_empty_component_set()
for component in left_over:   
    distance += np.sum(left_over)

In [ ]:
distance